In [ ]:
# ------------------------------------------------------------
# Please always run this cell first to set up the environment
# Especially important when running in Google Colab
# ------------------------------------------------------------

from pathlib import Path
import os

def in_colab():
    try:
        import google.colab  # noqa: F401
        return True
    except ImportError:
        return False


if in_colab():
    print("Running in Google Colab environment")

    REPO_NAME = "AI-Code"
    REPO_URL = "https://github.com/hku-kejintao/AI-Code.git"
    LESSON_DIR = "07 Case study of traffic prediction (with deep learning)/Demand-prediction"

    BASE_DIR = Path("/content")
    REPO_PATH = BASE_DIR / REPO_NAME
    LESSON_PATH = REPO_PATH / LESSON_DIR

    os.chdir(BASE_DIR)
    if not REPO_PATH.exists():
        print("Cloning repository...")
        !git clone {REPO_URL}
    else:
        print("Repository already exists, skip cloning.")

    if not LESSON_PATH.exists():
        available = [p.relative_to(REPO_PATH) for p in REPO_PATH.rglob("*") if p.is_dir()]
        raise FileNotFoundError(
            f"Lesson directory not found:\n  {LESSON_DIR}\n\n"
            f"Available directories under repo:\n  "
            + "\n  ".join(map(str, available[:20]))
        )
    os.chdir(LESSON_PATH)

    print(f"Current working directory: {os.getcwd()}")

else:
    print("Running in local / non-Colab environment")

# Exercise 07 Traffic Demand Prediction

In this exercise, you need to follow the requirements of each question to generate the Python code, and the following example is for reference：

- Sample Question: Write a program that takes the user's name as input and prints "Hello, [name]!" where [name] is the user's input.

- Potential Answer:

```python
    name = input("Enter your name: ")
    print("Hello, " + name + "!")
```
- If you enter 'David', the code will output 'Hello, David!', and this will satisfy the requirements.

## Attention
- Generally, there will be multiple answers for one question and you don't have to strictly follow the instructions in the tutorial, as long as you can make the output of the code meet the requirements of the question.
- If possible, strive to make your code concise and avoid excessive reliance on less commonly used libraries.
- You may need to search for information on the Internet to complete the excercise.

### Question 01: In this exercise, we use the same dataset as in the tutorial. The code to read the data and split it into training, validation, and test sets has been written for you. Run the following code to prepare the data. We only use the third region's data as dataset to run quickly. You can retrain the model on the full dataset if you want.

The feature set's shape should be `[Region, Sample, Features]`, the label set's shape should be `[Region, Sample]`. As we only use one region's data here, the first dim should be equal to 1 for both frature and label set.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

demand = np.load('./10min_demand_in_manhattan.npy')
demand = demand[2, :].reshape(1, -1)  # Remove this line to use the data of all regions if you want.
def construct_dataset(demand, hourly_trend=12, daily_trend=7, weekly_trend=4):
    X, y = [], []
    hour_range, day_range, week_range = 60 // 30, 24 * 60 // 30, 7 * 24 * 60 // 30
    for i in range(demand.shape[0]): # for each region
        X_1region, y_1region = [], []
        # here we leave 30 days for constructing the dataset
        for j in range(30*day_range, demand.shape[1]): # for each instance
            y_1region.append(demand[i, j])
            hour_trend = demand[i, j-hourly_trend*hour_range : j]
            day_trend = demand[i, list(range(j-daily_trend*day_range, j, day_range))]
            week_trend = demand[i, list(range(j-weekly_trend*week_range, j, week_range))]
            X_1region.append(np.concatenate([hour_trend, day_trend, week_trend]))
        X.append(X_1region)
        y.append(y_1region)
    X, y = np.array(X), np.array(y)
    return X, y

X, y = construct_dataset(demand, hourly_trend=12, daily_trend=7, weekly_trend=4)

day_range = 24 * 60 // 30
X_train, X_test = X[:, :-14*day_range, :], X[:, -14*day_range:, :]
y_train, y_test = y[:, :-14*day_range], y[:, -14*day_range:]

print('Training set size: ', X_train.shape, y_train.shape)  
print('Test set size: ', X_test.shape, y_test.shape)

### Question 02: Fit the traffic demand data using a full connection neural network on the training set. Make predictions using the test set and calculate the MAE, RMSE, and MAPE of the predictions. Output these metrics.

#### Question 02.1: The following code will help you convert data into a torch dataloader. Please set an appropriate batch size at the beginning, and then run the cell to get the `train_loader` and `test_loader`.

In [ ]:
batch_size =   # Choose a proper batch size here
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
def get_dataloader(X, y, device, bs, shuffle):
    return DataLoader(TensorDataset(torch.FloatTensor(X).to(device), torch.FloatTensor(y).to(device)), 
        batch_size=bs,
        shuffle=shuffle, 
        drop_last=False)
train_loader = get_dataloader(X_train.reshape(-1, X_test.shape[2]), y_train.reshape(-1,1), device, batch_size, True)
test_loader = get_dataloader(X_test.reshape(-1, X_test.shape[2]), y_test.reshape(-1,1), device, batch_size, False)

#### Question 02.2: Please write a training function, which should receive the model, optimizer, loss function, number of epochs, training data loader and test data loader as parameters, execute the training process, and output the average loss on the training set and test set after each epoch training.

In [3]:
def trainer(model, optimizer, loss_fn, epochs, train_loader, test_loader):
    

#### Question 02.3: Please design a full connection neural network, select the appropriate optimizer and loss function, and complete the training. 

You need to 
- Define the network structure;
- Select a loss function;
- Select an optimizer and corresponding learning rate;
- Train the model using the training function you defined in the previous question.

Some of the code has been written for you. 

**Try as many different network architectures as possible to find the best model.**

**Try to modify the parameters including learning rate, optimization algorithm, number of training epochs, etc. based on what you have learned previously to achieve better model performance.**

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        # write your code here
        
        # end of your code
        
    def forward(self, x):
        # write your code here
        
        # end of your code

input_size =   # Choose correct input size here
epochs =   # Choose proper number of epochs here
loss_fn =   # Choose proper loss function here
model = Net(input_size=input_size).to(device)
optimizer =   # Choose proper optimizer and learning rate here
trainer(model, optimizer, loss_fn, epochs, train_loader, test_loader)

#### Question 02.4: Combine all the data (training set and test set) into one dataset, use the trained model to make predictions, and plot the ground truth and your predictions on the same graph to show the performance of the model.

### Question 03: Fit the traffic demand data using a LSTM network on the training set. Make predictions using the test set and calculate the MAE, RMSE, and MAPE of the predictions. Output these metrics.

#### Question 03.1: The following code will help you convert the data into the data format required for the time series prediction task and build the torch dataloader. Please set an appropriate batch size and train/test ratio at the beginning, and then run the cell to get the `train_loader` and `test_loader`.

Like GRU in the tutorial, the feature set's shape for LSTM should be `[Sample, Series, Feature]`

In [ ]:
batch_size =   # Please choose a proper batch size here
train_test_ratio =   # Please choose a proper train test ratio here

timeseries = demand.reshape(-1, 1)
train_size = int(len(timeseries) * train_test_ratio)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]

def create_time_dataset(dataset, lookback=240):
    X, y = [], []
    for i in range(len(dataset)-lookback-2):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    X = torch.tensor(np.array(X)).to(device)
    y = torch.tensor(np.array(y)).to(device)
    y = y[:, -1, :]
    return X, y

train_X, train_y = create_time_dataset(train, lookback=240)
test_X, test_y = create_time_dataset(test, lookback=240)

print('Training set size: ', train_X.shape, train_y.shape)
print('Test set size: ', test_X.shape, test_y.shape)

train_loader = DataLoader(TensorDataset(train_X, train_y), shuffle=True, batch_size=batch_size)
test_loader = DataLoader(TensorDataset(test_X, test_y), shuffle=False, batch_size=batch_size)

#### Question 03.2: Please design a LSTM network, select the appropriate optimizer and loss function, and complete the training. 

You need to 
- Define the network structure;
- Select a loss function;
- Select an optimizer and corresponding learning rate;
- Train the model using the training function you defined in the previous question.

Some of the code has been written for you. 

**Try as many different network architectures as possible to find the best model.**

**Try to modify the parameters including learning rate, optimization algorithm, number of training epochs, etc. based on what you have learned previously to achieve better model performance.**

In [ ]:
class LstmModel(nn.Module):
    def __init__(self):
        super().__init__()
        # write your code here
        
        # end of your code

    def forward(self, x):
        # write your code here
        
        # end of your code


lr =   # Please choose a proper learning rate here
epochs =   # Please choose a proper number of epochs here
model = LstmModel().to(device)
optimizer =   # Choose a proper optimizer and learning rate here
loss_fn =   # Choose a proper loss function here
trainer(model, optimizer, loss_fn, epochs=epochs, train_loader=train_loader, test_loader=test_loader)

#### Question 03.3: Combine all the data (training set and test set) into one dataset, use the trained model to make predictions, and plot the ground truth and your predictions on the same graph to show the performance of the model.

### (Optional) Question 04: Predict the traffic demand data of all regions using a GCN network. Output the metrics. A NumPy file named "edges_GAman.npy" contains edge information of a road network.

The data of graph neural network includes node information and edge information. The shape of node information should be `[number of nodes, number of features on one node]`, and the shape of edge information should be `[2, num of edges]` in the form of an array of `(edge ​​starting point, edge end point)`. 

This task is a node prediction task, the label shape should be `[number of nodes, number of labels on one node]`.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

edge_index = np.load("./edges_GAman.npy")
traffic_data = np.load("./10min_demand_in_manhattan.npy")

num_regions, num_time_steps = traffic_data.shape

def create_graph_data(traffic_data, edge_index, history_steps=4):
    edge_index = torch.tensor(edge_index.T, dtype=torch.long)
    data_list = []
    for t in range(history_steps, num_time_steps):
        x = traffic_data[:, t-history_steps:t]
        x = torch.tensor(x, dtype=torch.float)
        y = traffic_data[:, t]
        y = torch.tensor(y, dtype=torch.float).reshape(-1, 1)
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)
    return data_list

history_steps = 20
data_list = create_graph_data(traffic_data, edge_index, history_steps)
dataset_size = len(data_list)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data_list, [train_size, test_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


for batch in train_loader:
    print(batch)
    print("Batch node feature shape:", batch.x.shape)
    print("Batch edge index shape:", batch.edge_index.shape)
    print("Batch target shape:", batch.y.shape)
    break

In [18]:
class TrafficGCN(torch.nn.Module):
    def __init__(self, input_dim=4, hidden_dim=16, output_dim=1):
        

    def forward(self, x, edge_index):
        
    

gcn = TrafficGCN(input_dim=history_steps).to(device)

In [19]:
def train_model(model, dataloader, optimizer, loss_fn, device):
    


def test_model(model, dataloader, device):
    

In [ ]:
lr = 
epochs = 
optimizer = 
loss_fn = 

for epoch in range(1, epochs + 1):
    train_loss = train_model(gcn, train_loader, optimizer, loss_fn, device)
    test_loss = test_model(gcn, test_loader, device)
    print(f"Epoch {epoch}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")